In [ ]:
pip install split_folders

In [ ]:
import splitfolders

In [ ]:
input_folder = "/content/drive/MyDrive/colab nootbook/ressnet/input_dataset"
output = "/content/drive/MyDrive/colab nootbook/ressnet/processed_data"
splitfolders.ratio(input_folder, output, seed=42, ratio=(.6, .2, .2))

Copying files: 10300 files [01:32, 111.22 files/s]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import scipy


from tensorflow.keras.preprocessing.image import load_img, img_to_array


from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, History
import pickle
import subprocess

In [ ]:
img_height,img_width=(224,224)
batch_size = 70

train_data_dir = r"/content/drive/MyDrive/colab nootbook/ressnet/processed data/train"
valid_data_dir = r"/content/drive/MyDrive/colab nootbook/ressnet/processed data/val"
test_data_dir = r"/content/drive/MyDrive/colab nootbook/ressnet/processed data/test"

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 3688 images belonging to 4 classes.
Found 816 images belonging to 4 classes.


In [ ]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    class_mode='categorical',
    subset='validation'
)

Found 816 images belonging to 4 classes.


In [ ]:
x,y = test_generator.next()
print(x,y.shape)

[[[[-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   ...
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]]

  [[-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   ...
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]]

  [[-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   ...
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]]

  ...

  [[-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   ...
   [-1.         -1.         -1.        ]
   [-1.         -1.         -1.        ]
   [-1.         

In [ ]:
base_model = VGG19(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint_loss = ModelCheckpoint('VGG19_model_loss.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1, save_weights_only=False)
model_checkpoint_accuracy = ModelCheckpoint('VGG19_model_accuracy.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1, save_weights_only=False)

history_callback = History()

#model.fit(train_generator, epochs=100, verbose = 2 )

history = model.fit(train_generator, epochs=100, verbose=2, validation_data=valid_generator, callbacks=[model_checkpoint_loss, model_checkpoint_accuracy, history_callback])

with open(' VGG19_training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Epoch 1/100

Epoch 1: val_loss improved from inf to 1.29957, saving model to VGG19_model_loss.h5

Epoch 1: val_accuracy improved from -inf to 0.27941, saving model to VGG19_model_accuracy.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


53/53 - 1765s - loss: 1.2630 - accuracy: 0.4170 - val_loss: 1.2996 - val_accuracy: 0.2794 - 1765s/epoch - 33s/step
Epoch 2/100

Epoch 2: val_loss improved from 1.29957 to 1.12441, saving model to VGG19_model_loss.h5

Epoch 2: val_accuracy improved from 0.27941 to 0.45956, saving model to VGG19_model_accuracy.h5
53/53 - 72s - loss: 1.0002 - accuracy: 0.5979 - val_loss: 1.1244 - val_accuracy: 0.4596 - 72s/epoch - 1s/step
Epoch 3/100

Epoch 3: val_loss improved from 1.12441 to 1.00600, saving model to VGG19_model_loss.h5

Epoch 3: val_accuracy improved from 0.45956 to 0.56618, saving model to VGG19_model_accuracy.h5
53/53 - 71s - loss: 0.8653 - accuracy: 0.6521 - val_loss: 1.0060 - val_accuracy: 0.5662 - 71s/epoch - 1s/step
Epoch 4/100

Epoch 4: val_loss did not improve from 1.00600

Epoch 4: val_accuracy did not improve from 0.56618
53/53 - 70s - loss: 0.7965 - accuracy: 0.6665 - val_loss: 1.0408 - val_accuracy: 0.5404 - 70s/epoch - 1s/step
Epoch 5/100

Epoch 5: val_loss improved from 1.

In [ ]:
model.save('VGG19._model.h5')